In [1]:
import os 
os.chdir('..')

In [2]:
import sys
import numpy as np 
import pandas as pd
sys.path.insert(0, '../') 
from source_code import data_loading 
from source_code import data_processing
from sklearn.model_selection import train_test_split

In [3]:
#load data for splitting
omic_dir_path = '../Downloaded_data'
gdsc2_target_path = 'data/GDSC2_Wed Aug GDSC2_30_15_49_31_2023.csv'
pubchem_ids_path = 'data/drugs_gdsc_to_pubID.csv'
rna, ic50, drugs_to_smiles = data_loading.load_omics_drugs_target(
    omic_dir_path, gdsc2_target_path, pubchem_ids_path)

drugs_with_smiles = drugs_to_smiles.index
rna.shape, ic50.shape, len(drugs_to_smiles)

drugs = np.zeros((len(drugs_with_smiles), len(drugs_with_smiles)))
#np.fill_diagonal(one_hot_drugs, 1)
drugs = pd.DataFrame(
    drugs, index=drugs_with_smiles, columns=drugs_with_smiles)

#creates df for each drug cl pair only for test train val split 
_, _, y = data_processing.create_all_drugs(rna, drugs, ic50)

c:\Users\Nik\Documents\PhD_code\year2_code\DRP-aberrations-and-comparisons\source_code\data_loading.py:133: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  cell_names_raw = pd.read_csv(f'{gdsc_dir_path}/gdsc_cell_names.csv', skiprows=1, skipfooter=1)


In [7]:
#cblind splits
num_pairs = {'train': [], 'val': [], 'test': []}
for seed in range(1, 4):
    split_type = 'c_blind'
    train_pairs, test_pairs, train_cls, test_cls = data_processing.cblind_split(
        seed, rna.index, drugs_with_smiles, y, train_size=0.8)
    print(' ')
    print('starting val split')
    print(' ')
    val_pairs, test_pairs, val_cls, test_cls = data_processing.cblind_split(
        seed, test_cls, drugs_with_smiles, test_pairs, train_size=0.5)

    save_path = f'data/train_test_pairs/{split_type}/'
    np.savetxt(f'{save_path}seed_{seed}_train', train_pairs, fmt='%s')
    np.savetxt(f'{save_path}seed_{seed}_val', val_pairs, fmt='%s')
    np.savetxt(f'{save_path}seed_{seed}_test', test_pairs, fmt='%s')

    num_pairs['train'].append(len(train_pairs))
    num_pairs['val'].append(len(val_pairs))
    num_pairs['test'].append(len(test_pairs))
    
num_pairs = pd.DataFrame(num_pairs)

Fraction of cls in sets, relative to all cls before mising values are removed
train fraction 0.7997787610619469, test fraction 0.2002212389380531
------
Fraction of cls in sets, relative to all cl drug pairs, after mising values are removed
train fraction 0.7216911944458025, test fraction 0.1810675695496993
 
starting val split
 
Fraction of cls in sets, relative to all cls before mising values are removed
train fraction 0.4972375690607735, test fraction 0.5027624309392266
------
Fraction of cls in sets, relative to all cl drug pairs, after mising values are removed
train fraction 0.45142089679802205, test fraction 0.4529165776380452
Fraction of cls in sets, relative to all cls before mising values are removed
train fraction 0.7997787610619469, test fraction 0.2002212389380531
------
Fraction of cls in sets, relative to all cl drug pairs, after mising values are removed
train fraction 0.7213183885004645, test fraction 0.18144037549503741
 
starting val split
 
Fraction of cls in sets, 

In [9]:
num_pairs

,train,val,test
0,118086,14789,14838
1,118025,14739,14949
2,118128,14576,15009


In [26]:
#mixed set splits
num_pairs = {'train': [], 'val': [], 'test': []}
for seed in range(1, 4):
    split_type = 'mixed'
    train_pairs, test_pairs, = train_test_split(
        y.index, random_state=seed, train_size=0.8)
    print(' ')
    print('starting val split')
    print(' ')
    val_pairs, test_pairs = data_processing.train_test_split(
        test_pairs, random_state=seed, test_size=0.5)
    
    assert set(train_pairs).intersection(val_pairs) == set()
    assert set(train_pairs).intersection(test_pairs) == set()
    assert set(test_pairs).intersection(val_pairs) == set()

    save_path = f'data/train_test_pairs/{split_type}/'
    np.savetxt(f'{save_path}seed_{seed}_train', train_pairs, fmt='%s')
    np.savetxt(f'{save_path}seed_{seed}_val', val_pairs, fmt='%s')
    np.savetxt(f'{save_path}seed_{seed}_test', test_pairs, fmt='%s')

    num_pairs['train'].append(len(train_pairs))
    num_pairs['val'].append(len(val_pairs))
    num_pairs['test'].append(len(test_pairs))
    
num_pairs = pd.DataFrame(num_pairs)

 
starting val split
 
 
starting val split
 
 
starting val split
 


In [11]:
num_pairs

,train,val,test
0,118170,14771,14772
1,118170,14771,14772
2,118170,14771,14772


In [12]:
#d blind split
num_pairs = {'train': [], 'val': [], 'test': []}
for seed in range(1, 4):
    split_type = 'd_blind'
    train_pairs, test_pairs, train_drugs, test_drugs = data_processing.dblind_split(
        seed, rna.index, drugs_with_smiles, y, train_size=0.8)
    print(' ')
    print('starting val split')
    print(' ')
    val_pairs, test_pairs, val_cls, test_cls = data_processing.dblind_split(
        seed, rna.index, test_drugs, test_pairs, train_size=0.5)

    save_path = f'data/train_test_pairs/{split_type}/'
    np.savetxt(f'{save_path}seed_{seed}_train', train_pairs, fmt='%s')
    np.savetxt(f'{save_path}seed_{seed}_val', val_pairs, fmt='%s')
    np.savetxt(f'{save_path}seed_{seed}_test', test_pairs, fmt='%s')

    num_pairs['train'].append(len(train_pairs))
    num_pairs['val'].append(len(val_pairs))
    num_pairs['test'].append(len(test_pairs))
    
num_pairs = pd.DataFrame(num_pairs)

Fraction of cls in sets, relative to all cls before mising values are removed
train fraction 0.7955801104972375, test fraction 0.20441988950276244
------
Fraction of cls in sets, relative to all cl drug pairs, after mising values are removed
train fraction 0.7200532929154647, test fraction 0.18270547108003715
 
starting val split
 
Fraction of cls in sets, relative to all cls before mising values are removed
train fraction 0.4864864864864865, test fraction 0.5135135135135135
------
Fraction of cls in sets, relative to all cl drug pairs, after mising values are removed
train fraction 0.411534321932552, test fraction 0.4822410906481703
Fraction of cls in sets, relative to all cls before mising values are removed
train fraction 0.7955801104972375, test fraction 0.20441988950276244
------
Fraction of cls in sets, relative to all cl drug pairs, after mising values are removed
train fraction 0.7143573069965287, test fraction 0.18840145699897326
 
starting val split
 
Fraction of cls in sets,

In [14]:
num_pairs

,train,val,test
0,117818,13765,16130
1,116886,14808,16019
2,118989,13684,15040
